<a href="https://colab.research.google.com/github/prayan2702/Backup-code/blob/main/Backtest_All_NSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install yfinance pandas numpy matplotlib

# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Upload the stock universe file
from google.colab import files
uploaded = files.upload() # Upload your 'NSE_EQ_ALL.csv' file here

# Load the stock symbols from the uploaded file
file_name = list(uploaded.keys())[0]
nse_data = pd.read_csv(file_name)
stock_universe = nse_data['Symbol'].dropna().tolist()

# Function to fetch historical data for a stock
def fetch_data(symbol, start_date="2010-01-01", end_date="2024-12-31"):
 try:
     df = yf.download(symbol + ".NS", start=start_date, end=end_date, progress=False)
     if not df.empty:
        df['Symbol'] = symbol
     return df
 except Exception as e:
     print(f"Failed to fetch data for {symbol}: {e}")
     return None

# Fetch data for all stocks in the universe
print("Fetching historical data...")
all_data = []
for symbol in stock_universe:
 data = fetch_data(symbol)
 if data is not None:
    all_data.append(data)

# Combine all data into a single DataFrame
if all_data:
 historical_data = pd.concat(all_data)
 historical_data.reset_index(inplace=True)
 print("Data fetching complete.")
else:
 print("No data fetched.")

# Calculate momentum for 3M, 6M, and 12M lookbacks
print("Calculating momentum...")
def calculate_momentum(df, lookback_period):
 return df['Close'] / df['Close'].shift(lookback_period) - 1

momentum_data = []
for symbol in stock_universe:
 stock_data = historical_data[historical_data['Symbol'] == symbol].copy()
 stock_data['3M_Momentum'] = calculate_momentum(stock_data, 63) # Approx. 3 months
 stock_data['6M_Momentum'] = calculate_momentum(stock_data, 126) # Approx. 6 months
 stock_data['12M_Momentum'] = calculate_momentum(stock_data, 252) # Approx. 12 months
 momentum_data.append(stock_data)

momentum_df = pd.concat(momentum_data)
momentum_df.reset_index(drop=True, inplace=True)

# Save momentum data for reference
momentum_df.to_csv("momentum_data.csv", index=False)
print("Momentum data saved to 'momentum_data.csv'.")

# Define pullback phases based on Nifty50 or any index
print("Identifying pullback phases...")
nifty_data = fetch_data("^NSEI") # Nifty50 Index
nifty_data['Drawdown'] = (nifty_data['Close'] / nifty_data['Close'].cummax()) - 1
pullback_phases = nifty_data[nifty_data['Drawdown'] < -0.1] # 10% or more decline

# Backtest performance during pullbacks
print("Backtesting during pullbacks...")
results = []
for lookback in ['3M_Momentum', '6M_Momentum', '12M_Momentum']:
 for date in pullback_phases.index:
  selected_stocks = momentum_df[
   (momentum_df['Date'] == date) &
   (momentum_df[lookback].rank(ascending=False) <= 10) # Top 10 stocks
]
  avg_return = selected_stocks['3M_Momentum'].mean() # Average return
  results.append({'Date': date, 'Lookback': lookback, 'AverageReturn': avg_return})

results_df = pd.DataFrame(results)
print("Backtesting complete.")

# Save results
results_df.to_csv("backtesting_results.csv", index=False)
print("Results saved to 'backtesting_results.csv'.")

# Plot results
print("Plotting results...")
results_df.groupby('Lookback')['AverageReturn'].plot(legend=True, title="Average Returns During Pullbacks")
plt.xlabel("Dates")
plt.ylabel("Average Return")
plt.show()


Saving NSE_EQ_ALL.csv to NSE_EQ_ALL.csv
Fetching historical data...


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GEOJIT-RE.NS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KALYANI.NS']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MITTAL-RE1.NS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
ERROR:yfinance:Failed to get ticker 'STEL.NS' reason: Expecting value: line 1 column 1 (char 0)
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['STEL.NS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
ERROR:yfinance:Failed to get ticker 'STERTOOLS.NS' reason: Expecting value: line 1 column 1 (char 0)
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['STERTOOLS.NS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
ERROR:yfinance:Failed to get ticker 'STLTECH.NS' reason: Expecting value: line 1 column 1 (char 0)
ERROR:yfina